In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/content/dataset_alpha_betha.csv') #Por favor ajustar la ruta del archivo al valor correspondiente dónde se ubique el archivo.

In [ ]:
df['Demand'] = pd.to_numeric(df['Demand'], errors='coerce')
df['Charges'] = pd.to_numeric(df['Charges'], errors='coerce')
df.isnull().sum()
df.dropna(inplace=True)

In [ ]:
X = df.drop(['Class', 'autoID'], axis=1)  # AutoID no es útil para predicción
y = df['Class']

In [ ]:
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)
numeric_features = ['Charges', 'Demand']
categorical_features = [col for col in X.columns if col not in numeric_features]


In [ ]:
# Exploramos y codificamos el dataset para las variables categóricas

X_encoded = X.copy()
label_encoders = {}

for col in categorical_features:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le
    print(f"   {col}: {len(le.classes_)} categorías únicas")

   SeniorCity: 2 categorías únicas
   Partner: 2 categorías únicas
   Dependents: 2 categorías únicas
   Service1: 2 categorías únicas
   Service2: 3 categorías únicas
   Security: 3 categorías únicas
   OnlineBackup: 3 categorías únicas
   DeviceProtection: 3 categorías únicas
   TechSupport: 3 categorías únicas
   Contract: 3 categorías únicas
   PaperlessBilling: 2 categorías únicas
   PaymentMethod: 4 categorías únicas


In [ ]:
X_encoded = pd.get_dummies(X_encoded, columns=categorical_features, drop_first=True)

In [ ]:
# Revisamos como nos quedó el dataset nuevo.

X_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7042
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Charges             7032 non-null   float64
 1   Demand              7032 non-null   float64
 2   SeniorCity_1        7032 non-null   bool   
 3   Partner_1           7032 non-null   bool   
 4   Dependents_1        7032 non-null   bool   
 5   Service1_1          7032 non-null   bool   
 6   Service2_1          7032 non-null   bool   
 7   Service2_2          7032 non-null   bool   
 8   Security_1          7032 non-null   bool   
 9   Security_2          7032 non-null   bool   
 10  OnlineBackup_1      7032 non-null   bool   
 11  OnlineBackup_2      7032 non-null   bool   
 12  DeviceProtection_1  7032 non-null   bool   
 13  DeviceProtection_2  7032 non-null   bool   
 14  TechSupport_1       7032 non-null   bool   
 15  TechSupport_2       7032 non-null   bool   
 16  Contract_1 

In [ ]:
# Partimos los datos en entrenamiento y prueba.

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded  # Mantiene proporciones de clases
)

In [ ]:
# Dado el desbalance entre las clase Alpha y Betha, hacemos un oversampling sobre el conjunto de entrenamiento para escalar los datos de Betha.

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
#Definimos un conjunto de modelos de base para explorar en un primer nivel como se comportan los datos según estos.

models = {
    'SVM': svm.SVC(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'KNN': KNeighborsClassifier(),
    'Logistic Regression': LogisticRegression()
}

In [ ]:
#Model Trainer nos permite probar múltiples modelos usando un pipeline.

class ModelTrainer:

  def __init__(self, models, X_train, y_train, X_test, y_test, components):
    self.models = models
    self.X_train = X_train
    self.y_train = y_train
    self.X_test = X_test
    self.y_test = y_test
    self.components = components

  def train_models(self):

    for name, model in self.models.items():

      pipeline = make_pipeline(
        #StandardScaler(),
        PCA(n_components=self.components),
        model
      )

      pipeline.fit(self.X_train, self.y_train)

      y_pred = pipeline.predict(self.X_test)

      print(f'Modelo: {name}')
      print(f'Precisión: {accuracy_score(self.y_test, y_pred)}')
      print(classification_report(self.y_test, y_pred))
      print(confusion_matrix(self.y_test, y_pred))
      print('-' * 50)


In [ ]:
ModelTrainer(models, X_train, y_train, X_test, y_test, 2).train_models()

Modelo: SVM
Precisión: 0.7341862117981521
              precision    recall  f1-score   support

           0       0.73      1.00      0.85      1033
           1       0.00      0.00      0.00       374

    accuracy                           0.73      1407
   macro avg       0.37      0.50      0.42      1407
weighted avg       0.54      0.73      0.62      1407

[[1033    0]
 [ 374    0]]
--------------------------------------------------
Modelo: Decision Tree
Precisión: 0.7114427860696517
              precision    recall  f1-score   support

           0       0.80      0.81      0.80      1033
           1       0.46      0.44      0.45       374

    accuracy                           0.71      1407
   macro avg       0.63      0.63      0.63      1407
weighted avg       0.71      0.71      0.71      1407

[[836 197]
 [209 165]]
--------------------------------------------------
Modelo: Random Forest
Precisión: 0.7583511016346838
              precision    recall  f1-score   su

Los resultados de los modelos anteriores no son muy precisos ni exhaustivos sobre todo para la clase Betha, cuyo desbalance es la 4ta parte de la clase Alpha.

In [ ]:
#Repetimos el mismo procedimiento haciendo análisis de componentes.

components = [3, 5, 7, 12]

for comp in components:
  print(f'Componentes: {comp}')
  ModelTrainer(models, X_train, y_train, X_test, y_test, comp).train_models()
  print('-' * 50)

Componentes: 3
Modelo: SVM
Precisión: 0.7341862117981521
              precision    recall  f1-score   support

           0       0.73      1.00      0.85      1033
           1       0.00      0.00      0.00       374

    accuracy                           0.73      1407
   macro avg       0.37      0.50      0.42      1407
weighted avg       0.54      0.73      0.62      1407

[[1033    0]
 [ 374    0]]
--------------------------------------------------
Modelo: Decision Tree
Precisión: 0.7157071783937455
              precision    recall  f1-score   support

           0       0.81      0.81      0.81      1033
           1       0.47      0.47      0.47       374

    accuracy                           0.72      1407
   macro avg       0.64      0.64      0.64      1407
weighted avg       0.72      0.72      0.72      1407

[[833 200]
 [200 174]]
--------------------------------------------------
Modelo: Random Forest
Precisión: 0.7690120824449183
              precision    recall

De nuevo los resultados anteriores muestran una precisión y un recall muy bajo.

In [ ]:
pip install xgboost
pip install lightgbm
pip install catboost

In [ ]:
# Repitamos el ejercicio con modelos mejor adaptados para clases desbalanceadas.

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [ ]:
models = {
    'XGBoost': XGBClassifier(use_label_encoders=False, eval_metric='logloss', random_state=42),
    'LightGBM': LGBMClassifier(random_state=42),
    'CatBoost': CatBoostClassifier(verbose=0, random_seed=42)
}

In [ ]:
ModelTrainer(models, X_train, y_train, X_test, y_test, 5).train_models()

Modelo: XGBoost
Precisión: 0.775408670931059
              precision    recall  f1-score   support

           0       0.83      0.87      0.85      1033
           1       0.59      0.51      0.54       374

    accuracy                           0.78      1407
   macro avg       0.71      0.69      0.70      1407
weighted avg       0.77      0.78      0.77      1407

[[902 131]
 [185 189]]
--------------------------------------------------
[LightGBM] [Info] Number of positive: 1495, number of negative: 4130
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000575 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 5625, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265778 -> initscore=-1.016151
[LightGBM] [Info] Start training from score -1.016151
Modelo: LightGBM
Precisión: 0.7853589196872779
              pre

In [ ]:
for comp in components:
  print(f'Componentes: {comp}')
  ModelTrainer(models, X_train, y_train, X_test, y_test, comp).train_models()
  print('-' * 50)

Componentes: 3
Modelo: XGBoost
Precisión: 0.7647476901208244
              precision    recall  f1-score   support

           0       0.82      0.87      0.84      1033
           1       0.57      0.48      0.52       374

    accuracy                           0.76      1407
   macro avg       0.69      0.67      0.68      1407
weighted avg       0.75      0.76      0.76      1407

[[896 137]
 [194 180]]
--------------------------------------------------
[LightGBM] [Info] Number of positive: 1495, number of negative: 4130
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000371 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 5625, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265778 -> initscore=-1.016151
[LightGBM] [Info] Start training from score -1.016151
Modelo: LightGBM
Precisión: 0.7796730632551528
  

Vamos a repetir el proceso, pero ahora haciendo búsqueda más intensiva de hiperparámetros con GridSearch y validación cruzada.

In [ ]:
param_grids = {
    "XGBoost": {
        'n_estimators': [200, 400],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.05, 0.1],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'scale_pos_weight': [2.76]
    },
    "LightGBM": {
        'n_estimators': [200, 400],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.05, 0.1],
        'num_leaves': [15, 31, 63],
        'subsample': [0.8, 1.0],
        'scale_pos_weight': [None, 2.76]
    },
    "CatBoost": {
        'iterations': [200, 400],
        'depth': [4, 6, 8],
        'learning_rate': [0.01, 0.05, 0.1],
        'l2_leaf_reg': [1, 3, 5],
        'bagging_temperature': [0, 0.5, 1],
        'class_weights': [[1, 2.76]]
    }
}


In [ ]:
best_models = {}
for name, model in models.items():
  print(f'Modelo: {name}')

  grid_search = GridSearchCV(
      estimator=model,
      param_grid=param_grids[name],
      cv=5,
      n_jobs=-1,
      verbose=2
  )

  grid_search.fit(X_train, y_train)

  best_models[name] = grid_search.best_estimator_

Modelo: XGBoost
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Modelo: LightGBM
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[LightGBM] [Info] Number of positive: 1495, number of negative: 4130
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001330 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 550
[LightGBM] [Info] Number of data points in the train set: 5625, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265778 -> initscore=-1.016151
[LightGBM] [Info] Start training from score -1.016151
Modelo: CatBoost
Fitting 5 folds for each of 162 candidates, totalling 810 fits


In [ ]:
#Encontramos los mejores modelos.

print("\n Resultados finales:\n")
for name, model in best_models.items():
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    print(classification_report(y_test, preds))
    print(f"{name}: {acc:.4f}")


📊 Resultados finales:

              precision    recall  f1-score   support

           0       0.88      0.76      0.81      1033
           1       0.52      0.71      0.60       374

    accuracy                           0.75      1407
   macro avg       0.70      0.74      0.71      1407
weighted avg       0.78      0.75      0.76      1407

XGBoost: 0.7463
              precision    recall  f1-score   support

           0       0.83      0.89      0.86      1033
           1       0.63      0.51      0.56       374

    accuracy                           0.79      1407
   macro avg       0.73      0.70      0.71      1407
weighted avg       0.78      0.79      0.78      1407

LightGBM: 0.7903
              precision    recall  f1-score   support

           0       0.89      0.76      0.82      1033
           1       0.53      0.73      0.61       374

    accuracy                           0.75      1407
   macro avg       0.71      0.75      0.71      1407
weighted avg     

CatBoost es el que presenta mejor rendimiento, por lo cual intentaremos optimizarlo más con optuna.

In [ ]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 6.6 MB/s eta 0:00:00


In [ ]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer

def objective(trial):
    # Definir el espacio de búsqueda
    params = {
        'iterations': trial.suggest_int('iterations', 200, 800),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'class_weights': [1, 2.76],
        'random_strength': trial.suggest_float('random_strength', 0, 2),
        'verbose': 0
    }

    model = CatBoostClassifier(**params)

    # Evaluamos con validación cruzada
    score = cross_val_score(
        model, X_train, y_train,
        scoring=make_scorer(f1_score),
        cv=3,
        n_jobs=-1
    ).mean()

    return score

# Crear y ejecutar el estudio
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Mejores parámetros:")
print(study.best_params)
print("Mejor F1:", study.best_value)


[I 2025-11-05 01:02:09,742] A new study created in memory with name: no-name-6f4590d6-2a1a-436d-95f6-666692d6be0a
[I 2025-11-05 01:02:51,811] Trial 0 finished with value: 0.6256717709564651 and parameters: {'iterations': 342, 'depth': 10, 'learning_rate': 0.013821487217027572, 'l2_leaf_reg': 7.676607642828506, 'bagging_temperature': 0.5234666616824045, 'border_count': 131, 'random_strength': 1.9996877413789191}. Best is trial 0 with value: 0.6256717709564651.
[I 2025-11-05 01:03:24,657] Trial 1 finished with value: 0.621483454939223 and parameters: {'iterations': 781, 'depth': 9, 'learning_rate': 0.015319579043164751, 'l2_leaf_reg': 4.567529624042563, 'bagging_temperature': 0.2802029426159457, 'border_count': 66, 'random_strength': 0.716323724133245}. Best is trial 0 with value: 0.6256717709564651.
[I 2025-11-05 01:03:32,654] Trial 2 finished with value: 0.621364849401873 and parameters: {'iterations': 404, 'depth': 7, 'learning_rate': 0.023118018681818255, 'l2_leaf_reg': 8.56205905048

Mejores parámetros:
{'iterations': 543, 'depth': 5, 'learning_rate': 0.012629950316874172, 'l2_leaf_reg': 7.417537467606664, 'bagging_temperature': 0.58685530200088, 'border_count': 140, 'random_strength': 0.9019333926811216}
Mejor F1: 0.6358130374717383


In [ ]:
optuna.visualization.plot_optimization_history(study)
optuna.visualization.plot_param_importances(study)


In [ ]:
print("Mejor F1-score:", study.best_value)
print("Mejores parámetros:")
for k, v in study.best_params.items():
    print(f"  {k}: {v}")


🏆 Mejor F1-score: 0.6358130374717383
🧩 Mejores parámetros:
  iterations: 543
  depth: 5
  learning_rate: 0.012629950316874172
  l2_leaf_reg: 7.417537467606664
  bagging_temperature: 0.58685530200088
  border_count: 140
  random_strength: 0.9019333926811216


Ajustamos de nuevo el modelo con los parámetros encontrados por optuna.

In [ ]:
best_params = study.best_params
best_model = CatBoostClassifier(**best_params)
best_model.fit(X_train, y_train)

In [ ]:
y_pred = best_model.predict(X_test > )
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.90      0.87      1033
           1       0.66      0.52      0.58       374

    accuracy                           0.80      1407
   macro avg       0.75      0.71      0.73      1407
weighted avg       0.79      0.80      0.79      1407



In [ ]:
y_pred_adj = (y_pred > 0.8).astype(int)
print(classification_report(y_test, y_pred_adj))

              precision    recall  f1-score   support

           0       0.84      0.90      0.87      1033
           1       0.66      0.52      0.58       374

    accuracy                           0.80      1407
   macro avg       0.75      0.71      0.73      1407
weighted avg       0.79      0.80      0.79      1407



In [ ]:
print(confusion_matrix(y_test, y_pred_adj))

[[933 100]
 [179 195]]


Análisis de rendimiento de modelos:

- De acuerdo con los Score, el mejor modelo fué CatBoost. sin embargo, la precisción del modelo y recall continuan siendo bastante bajos. Esto debe deberse en mayor efecto por el desbalance drástico de las clases.  y/o por que puede ser que las variables de entrada no sean los suficientemente representativas de la variable de salida.

- Otras alternativas a explorar pueden ser redes neuronales, sin embargo el conjunto de datos tampoco parece ser lo suficientemente grande para lograr este objetivo. sin embargo para este ejercicio, optaremos por quedarnos con CatBoost con los mejores resultados.
